In [1]:
# Import dependencies & display configuration
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.parse import urlparse
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', 50)

In [ ]:
# Data wrangling and featurization
pageviews = pd.read_csv("~/Desktop/data/pageviews.csv", 
                        index_col=0, 
                        parse_dates=[2]) \
    .rename(columns={'time': 'created_at'})
users = pd.read_csv("~/Desktop/data/users.csv", 
                    index_col=0, 
                    parse_dates=[2])

assert pageviews.user_id.nunique() == users.user_id.nunique()